# Function Calling with the Azure AI Agent Service 

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from user_functions import user_functions
from pprint import pformat



project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [4]:
with project_client:
    # Initialize agent toolset with user functions and code interpreter
    # [START create_agent_toolset]
    functions = FunctionTool(user_functions)
    code_interpreter = CodeInterpreterTool()

    toolset = ToolSet()
    toolset.add(functions)
    toolset.add(code_interpreter)

    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-assistant",
        instructions="You are a helpful assistant",
        toolset=toolset,
    )
    # [END create_agent_toolset]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, send an email with the datetime and weather information in New York?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    # [START create_and_process_run]
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id, assistant_id=agent.id)
    # [END create_and_process_run]
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("\n=== Thread Messages ===")
    print(pformat(messages.__dict__, indent=2, width=80))

Created agent, ID: asst_VXEukXmT3n6LF0t1bhc2SV9d
Created thread, ID: thread_ZuQgoIwa16PlKGuSA2ebF1Gu
Created message, ID: msg_iW5omVb3C2A4djOMMUPnRHwu
Sending email to example@example.com...
Subject: Current Datetime and Weather in New York
Body:
The current datetime is: 2025-02-21 15:18:45

The current weather in New York is: Sunny, 25°C
Run finished with status: RunStatus.COMPLETED
Deleted agent

=== Thread Messages ===
{ '_data': { 'data': [ {'id': 'msg_3Ww3eQWkC2IvfFhseQD4X8rs', 'object': 'thread.message', 'created_at': 1740151128, 'assistant_id': 'asst_VXEukXmT3n6LF0t1bhc2SV9d', 'thread_id': 'thread_ZuQgoIwa16PlKGuSA2ebF1Gu', 'run_id': 'run_nNf3xFLdFKZtVXe60H8RBINL', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "The email with the current datetime and weather information in New York has been successfully sent. \n\nHere's what was included in the email:\n- **Datetime:** 2025-02-21 15:18:45\n- **Weather:** Sunny, 25°C \n\nIf you need anything else, feel free t